In [1]:
import pandas as pd
import re
import os
import time
import random
import numpy as np

try:
  %tensorflow_version 2.x # enable TF 2.x in Colab
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import pickle

import spacy

nlp = spacy.load("en_core_web_sm")
from nltk.translate.bleu_score import corpus_bleu

In [2]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [3]:
with open('D:\MATH_AI\Project\MAWPS_Augmented.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
df.shape

(38144, 2)

In [5]:
df.head(20)

,Question,Equation
963,A painter needed to paint 12 rooms in a build...,X=(7.0*(12.0-5.0))
3897,Brenda had 253 raspberry. John gripped some ra...,X = 253 - 66
27626,Casey wants to share some Bread among 17 frien...,X = 39 * 17
32530,Liza had 34 Press. Thomas furnished him some m...,X = 64 - 34
16266,George wants to distribute 125 mangos among 25...,X = 125 / 25
21122,Richard has 148 Marbles. Gary gave him 42 more...,X = 42 + 148
17377,Mary wants to impart 547 limes among 24 friend...,X = 547 / 24
20997,Juana had some apricot. He hash each apricot i...,X = 149 / 9
2745,Timmy had 86 watermelon. Kandi took 40 from hi...,X = 86 - 40
2080,William had 99 pear. Martin took 67 from him. ...,X = 99 - 67


In [6]:
X = list(df['Question'].values)

In [7]:
def spacify(s):
    return ' '.join(list(s))

In [8]:
Y = list(df['Equation'].apply(lambda y: spacify(y)).values)

In [9]:
print(X[:10])
Y[:10]

[' A painter needed to paint 12 rooms in a building. Each room takes 7 hours to paint. If he already painted 5 rooms, how much longer will he take to paint the rest? ', 'Brenda had 253 raspberry. John gripped some raspberry. Now Brenda has 66  raspberry. How many did John grippeds?', 'Casey wants to share some Bread among 17 friends.If each friend get 39 Bread, then how many Bread john would have?', 'Liza had 34 Press. Thomas furnished him some more. Now Liza has 64 Press. How many did Thomas furnish him?', 'George wants to distribute 125 mangos among 25 friends. How many would each friend acquire?', 'Richard has 148 Marbles. Gary gave him 42 more. How many Marbles does Richard have altogether?', 'Mary wants to impart 547 limes among 24 friends. How many would each friend obtain?', 'Juana had some apricot. He hash each apricot into 9 slices. If total 149 apricot slices Juana make, then how many apricot Juana had?', 'Timmy had 86 watermelon. Kandi took 40 from him. Now How many watermel

['X = ( 7 . 0 * ( 1 2 . 0 - 5 . 0 ) )',
 'X   =   2 5 3   -   6 6',
 'X   =   3 9   *   1 7',
 'X   =   6 4   -   3 4',
 'X   =   1 2 5   /   2 5',
 'X   =   4 2   +   1 4 8',
 'X   =   5 4 7   /   2 4',
 'X   =   1 4 9   /   9',
 'X   =   8 6   -   4 0',
 'X   =   9 9   -   6 7']

In [10]:
def preprocess_X(s):
    s = s.lower().strip()
    s = re.sub(r"([?.!,’])", r" \1 ", s)
    s = re.sub(r"([0-9])", r" \1 ", s)
    s = re.sub(r'[" "]+', " ", s)
    s = s.rstrip().strip()
    return s

def preprocess_Y(e):
    e = e.lower().strip()
    return e

In [11]:
X_pp = list(map(preprocess_X, X))
Y_pp = list(map(preprocess_Y, Y))

In [12]:
X_pp[:10]

['a painter needed to paint 1 2 rooms in a building . each room takes 7 hours to paint . if he already painted 5 rooms , how much longer will he take to paint the rest ?',
 'brenda had 2 5 3 raspberry . john gripped some raspberry . now brenda has 6 6 raspberry . how many did john grippeds ?',
 'casey wants to share some bread among 1 7 friends . if each friend get 3 9 bread , then how many bread john would have ?',
 'liza had 3 4 press . thomas furnished him some more . now liza has 6 4 press . how many did thomas furnish him ?',
 'george wants to distribute 1 2 5 mangos among 2 5 friends . how many would each friend acquire ?',
 'richard has 1 4 8 marbles . gary gave him 4 2 more . how many marbles does richard have altogether ?',
 'mary wants to impart 5 4 7 limes among 2 4 friends . how many would each friend obtain ?',
 'juana had some apricot . he hash each apricot into 9 slices . if total 1 4 9 apricot slices juana make , then how many apricot juana had ?',
 'timmy had 8 6 water

In [13]:
Y_pp[:10]

['x = ( 7 . 0 * ( 1 2 . 0 - 5 . 0 ) )',
 'x   =   2 5 3   -   6 6',
 'x   =   3 9   *   1 7',
 'x   =   6 4   -   3 4',
 'x   =   1 2 5   /   2 5',
 'x   =   4 2   +   1 4 8',
 'x   =   5 4 7   /   2 4',
 'x   =   1 4 9   /   9',
 'x   =   8 6   -   4 0',
 'x   =   9 9   -   6 7']

In [14]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    return tensor, lang_tokenizer

In [15]:
X_tensor, X_lang_tokenizer = tokenize(X_pp)

In [16]:
len(X_lang_tokenizer.word_index)

5887

In [17]:
Y_tensor, Y_lang_tokenizer = tokenize(Y_pp)

In [18]:
len(Y_lang_tokenizer.word_index)

45

In [19]:
previous_length = len(Y_lang_tokenizer.word_index)

In [20]:
def append_head_tail(x, last_int):
    l = []
    l.append(last_int + 1)
    l.extend(x)
    l.append(last_int + 2)
    return l

In [21]:
X_tensor_list = [append_head_tail(i, len(X_lang_tokenizer.word_index)) for i in X_tensor]
Y_tensor_list = [append_head_tail(i, len(Y_lang_tokenizer.word_index)) for i in Y_tensor]

In [22]:
X_tensor = tf.keras.preprocessing.sequence.pad_sequences(X_tensor_list, padding='post')
Y_tensor = tf.keras.preprocessing.sequence.pad_sequences(Y_tensor_list, padding='post')

In [23]:
X_tensor

array([[5888,  104, 2454, ...,    0,    0,    0],
       [5888,  280,    6, ...,    0,    0,    0],
       [5888,  811,   43, ...,    0,    0,    0],
       ...,
       [5888,  380,   12, ...,    0,    0,    0],
       [5888,  167,    6, ...,    0,    0,    0],
       [5888,  404,    6, ...,    0,    0,    0]])

In [24]:
Y_tensor

array([[46,  2,  1, ...,  0,  0,  0],
       [46,  2,  1, ...,  0,  0,  0],
       [46,  2,  1, ...,  0,  0,  0],
       ...,
       [46,  2,  1, ...,  0,  0,  0],
       [46,  2,  1, ...,  0,  0,  0],
       [46,  2,  1, ...,  0,  0,  0]])

In [25]:
keys = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
        '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
        '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
        '46', '47', '48', '49', '50']

for idx,key in enumerate(keys):
    Y_lang_tokenizer.word_index[key] = len(Y_lang_tokenizer.word_index) + idx + 4

In [26]:
len(Y_lang_tokenizer.word_index)

86

In [27]:
X_tensor_train, X_tensor_test, Y_tensor_train, Y_tensor_test = train_test_split(X_tensor, Y_tensor, test_size=0.05, random_state=42)

In [28]:
print(len(X_tensor_train), len(X_tensor_test), len(Y_tensor_train), len(Y_tensor_test))

36236 1908 36236 1908


In [29]:
TRAINING_SET_SIZE = len(X_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = np.floor(TRAINING_SET_SIZE/BATCH_SIZE)

data = tf.data.Dataset.from_tensor_slices((X_tensor_train, Y_tensor_train)).shuffle(TRAINING_SET_SIZE)
data = data.batch(BATCH_SIZE, drop_remainder=True)

num_layers = 4
d_model = 128       # Embedding dimension
dff = 512           # Dimensionality of inner-layer of FNN
num_heads = 8       # Number of parallel attention layers (heads)
dropout_rate = 0

X_vocabulary_size = len(X_lang_tokenizer.word_index) + 3
Y_vocabulary_size = len(Y_lang_tokenizer.word_index) + 3

In [30]:
data = data.prefetch(tf.data.experimental.AUTOTUNE)

In [31]:
data

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 124), dtype=tf.int32, name=None), TensorSpec(shape=(64, 66), dtype=tf.int32, name=None))>

In [32]:
X_batch_example, Y_batch_example = next(iter(data))

print(X_batch_example, Y_batch_example)

tf.Tensor(
[[5888  820    6 ...    0    0    0]
 [5888  291    6 ...    0    0    0]
 [5888   98    6 ...    0    0    0]
 ...
 [5888  485    6 ...    0    0    0]
 [5888  385    6 ...    0    0    0]
 [5888 3553   43 ...    0    0    0]], shape=(64, 124), dtype=int32) tf.Tensor(
[[46  2  1 ...  0  0  0]
 [46  2  1 ...  0  0  0]
 [46  2  1 ...  0  0  0]
 ...
 [46  2  1 ...  0  0  0]
 [46  2  1 ...  0  0  0]
 [46  2  1 ...  0  0  0]], shape=(64, 66), dtype=int32)


In [33]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [34]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [35]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [36]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask


In [37]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [38]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                    (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [39]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [40]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        # normalize data per feature instead of batch
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        # Multi-head attention layer
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        # add residual connection to avoid vanishing gradient problem
        out1 = self.layernorm1(x + attn_output)

        # Feedforward layer
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        # add residual connection to avoid vanishing gradient problem
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

In [41]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding,
                                                self.d_model)

        # Create encoder layers (count: num_layers)
        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                        for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x

In [42]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)


    def call(self, x, enc_output, training,
            look_ahead_mask, padding_mask):

        # Masked multihead attention layer (padding + look-ahead)
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        # again add residual connection
        out1 = self.layernorm1(attn1 + x)

        # Masked multihead attention layer (only padding)
        # with input from encoder as Key and Value, and input from previous layer as Query
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        # again add residual connection
        out2 = self.layernorm2(attn2 + out1)

        # Feedforward layer
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        # again add residual connection
        out3 = self.layernorm3(ffn_output + out2)
        return out3, attn_weights_block1, attn_weights_block2

In [43]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
                maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        # Create decoder layers (count: num_layers)
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                        for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,
            look_ahead_mask, padding_mask):

        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)

        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))

        x += self.pos_encoding[:,:seq_len,:]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                                look_ahead_mask, padding_mask)

        # store attenion weights, they can be used to visualize while translating
        attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
        attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        return x, attention_weights

In [44]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
                target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                            input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                            target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask,
            look_ahead_mask, dec_padding_mask):

        # Pass the input to the encoder
        enc_output = self.encoder(inp, training, enc_padding_mask)

        # Pass the encoder output to the decoder
        dec_output, attention_weights = self.decoder(
            tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        # Pass the decoder output to the last linear layer
        final_output = self.final_layer(dec_output)

        return final_output, attention_weights

In [45]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        d_model = tf.cast(self.d_model, tf.float32)  # Ensure d_model is tf.float32
        arg1 = tf.math.rsqrt(tf.cast(step, tf.float32))  # Cast step to tf.float32
        arg2 = tf.cast(step, tf.float32) * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(d_model) * tf.math.minimum(arg1, arg2)

In [46]:
d_model = 512  # Set the value of d_model here
learning_rate = CustomSchedule(d_model)

# Adam optimizer with a custom learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [47]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [48]:
def loss_function(real, pred):
    # Apply a mask to paddings (0)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    accuracies = tf.equal(tf.cast(real, dtype=tf.int32), tf.cast(tf.argmax(pred, axis=2), dtype=tf.int32))

    mask = tf.math.logical_not(tf.math.equal(tf.cast(real, dtype=tf.int32), 0))
    accuracies = tf.math.logical_and(mask, accuracies)

    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [49]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
train_accuracy_mean = tf.keras.metrics.Mean(name='train_accuracy_mean')

In [50]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          X_vocabulary_size, Y_vocabulary_size,
                          pe_input=X_vocabulary_size,
                          pe_target=Y_vocabulary_size,
                          rate=dropout_rate)

In [51]:
def create_masks(inp, tar):
    # Encoder padding mask (Used in the 2nd attention block in the decoder too.)
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    # Look ahead mask (for hiding the rest of the sequence in the 1st decoder attention layer)
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, look_ahead_mask, dec_padding_mask

In [52]:
drive_root = 'D:/MATH_AI/Project'

In [53]:
checkpoint_directory = os.path.join(drive_root, "MLProject/checkpoints")
checkpoint_directory = os.path.join(checkpoint_directory, "training_checkpoints/moops_transfomer")

print("Checkpoints directory is", checkpoint_directory)
if os.path.exists(checkpoint_directory):
    print("Checkpoints folder already exists!")
else:
    print("Creating a checkpoints directory...")
    os.makedirs(checkpoint_directory)


checkpoint = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_directory, max_to_keep=None)

Checkpoints directory is D:/MATH_AI/Project\MLProject/checkpoints\training_checkpoints/moops_transfomer
Checkpoints folder already exists!


In [54]:
last_checkpoint = ckpt_manager.latest_checkpoint
last_checkpoint

'D:/MATH_AI/Project\\MLProject/checkpoints\\training_checkpoints/moops_transfomer\\ckpt-5'

In [55]:
if last_checkpoint:
    current_epoch_num = int(last_checkpoint.split('/')[-1].split('-')[-1])
    checkpoint.restore(last_checkpoint)
    print('Last checkpoint has been restored!')
else:
    current_epoch_num = 0

Last checkpoint has been restored!


In [56]:
print(current_epoch_num)

5


# ***TRAINING***

In [57]:
EPOCHS = 17

In [58]:
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp, tar_inp,
                                    True,
                                    enc_padding_mask,
                                    look_ahead_mask,
                                    dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)
    train_accuracy_mean(accuracy_function(tar_real, predictions))

In [59]:
'''for epoch in range(current_epoch_num, EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    train_accuracy_mean.reset_states()

    # inp -> math problem, tar -> expression
    for (batch, (inp, tar)) in enumerate(data):
        train_step(inp, tar)

        if batch % 50 == 0:
            print (f'Epoch {epoch + 1}, Batch {batch}, Loss {train_loss.result():.5f},\
             SC Accuracy {train_accuracy.result():.5f}, Mean Accuracy {train_accuracy_mean.result():.5f}')

    ckpt_save_path = ckpt_manager.save()
    print (f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

    print (f'Epoch {epoch + 1}, Loss {train_loss.result():.5f},\
     SC Accuracy {train_accuracy.result():.5f}, Mean Accuracy {train_accuracy_mean.result():.5f}')

    print (f'Training time for this epoch: {(time.time() - start):.5f} seconds\n')'''

"for epoch in range(current_epoch_num, EPOCHS):\n    start = time.time()\n\n    train_loss.reset_states()\n    train_accuracy.reset_states()\n    train_accuracy_mean.reset_states()\n\n    # inp -> math problem, tar -> expression\n    for (batch, (inp, tar)) in enumerate(data):\n        train_step(inp, tar)\n\n        if batch % 50 == 0:\n            print (f'Epoch {epoch + 1}, Batch {batch}, Loss {train_loss.result():.5f},             SC Accuracy {train_accuracy.result():.5f}, Mean Accuracy {train_accuracy_mean.result():.5f}')\n\n    ckpt_save_path = ckpt_manager.save()\n    print (f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')\n\n    print (f'Epoch {epoch + 1}, Loss {train_loss.result():.5f},     SC Accuracy {train_accuracy.result():.5f}, Mean Accuracy {train_accuracy_mean.result():.5f}')\n\n    print (f'Training time for this epoch: {(time.time() - start):.5f} seconds\n')"

In [60]:
MAX_LENGTH = 40

In [61]:
def evaluate(input_problem):
    start_token = [len(X_lang_tokenizer.word_index)+1]
    end_token = [len(X_lang_tokenizer.word_index)+2]

    # input_problem is the word problem, hence adding the start and end token
    input_problem = start_token + [X_lang_tokenizer.word_index[i] for i in preprocess_X(input_problem).split(' ')] + end_token
    encoder_input = tf.expand_dims(input_problem, 0)

    # start with expression's start token
    decoder_input = [previous_length+1]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(MAX_LENGTH):
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(encoder_input,
                                                    output,
                                                    False,
                                                    enc_padding_mask,
                                                    look_ahead_mask,
                                                    dec_padding_mask)

        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), dtype=tf.int32)

        # return the result if the predicted_id is equal to the end token
        if predicted_id == previous_length + 2:
            return tf.squeeze(output, axis=0), attention_weights

        # concatenate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)
    return tf.squeeze(output, axis=0), attention_weights

In [62]:
def plot_attention_weights(attention, problem, result, layer):
    fig = plt.figure(figsize=(8, 16))

    sentence = preprocess_X(problem)

    attention = tf.squeeze(attention[layer], axis=0)

    for head in range(attention.shape[0]):
        ax = fig.add_subplot(4, 2, head+1)

        # plot the attention weights
        ax.matshow(attention[head][:-1, :], cmap='viridis')

        fontdict = {'fontsize': 11}

        ax.set_xticks(range(len(sentence.split(' '))+2))
        ax.set_yticks(range(len([Y_lang_tokenizer.index_word[i] for i in list(result.numpy())
                            if i < len(Y_lang_tokenizer.word_index) and i not in [0,previous_length+1,previous_length+2]])+3))


        ax.set_ylim(len([Y_lang_tokenizer.index_word[i] for i in list(result.numpy())
                            if i < len(Y_lang_tokenizer.word_index) and i not in [0,previous_length+1,previous_length+2]]), -0.5)

        ax.set_xticklabels(
            ['<start>']+sentence.split(' ')+['<end>'],
            fontdict=fontdict, rotation=90)

        ax.set_yticklabels([Y_lang_tokenizer.index_word[i] for i in list(result.numpy())
                            if i < len(Y_lang_tokenizer.word_index) and i not in [0,previous_length+1,previous_length+2]],
                        fontdict=fontdict)

        ax.set_xlabel(f'Head {head+1}')

    plt.tight_layout()
    plt.show()

In [63]:
def solve(problem, plot='', plot_Attention_Weights=False):
    prediction, attention_weights = evaluate(problem)
    predicted_expression = [Y_lang_tokenizer.index_word[i] \
                          for i in list(prediction.numpy()) \
                          if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
    input_text = problem
    predicted_translation = ' '.join(predicted_expression)

    print(f'Input: {input_text}')
    print(f'Predicted translation: {predicted_translation}')



    if plot_Attention_Weights:
        plot_attention_weights(attention_weights, problem, prediction, plot)

    return

In [64]:
def evaluate_testset(input_problem):
    start_token = [len(X_lang_tokenizer.word_index)+1]
    end_token = [len(X_lang_tokenizer.word_index)+2]

    # input_problem is the word problem, hence adding the start and end token
    input_problem = start_token + list(input_problem.numpy()[0]) + end_token
    encoder_input = tf.expand_dims(input_problem, 0)

    # start with expression's start token
    decoder_input = [previous_length+1]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(MAX_LENGTH):
        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(encoder_input,
                                                    output,
                                                    False,
                                                    enc_padding_mask,
                                                    look_ahead_mask,
                                                    dec_padding_mask)

        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), dtype=tf.int32)

        # return the result if the predicted_id is equal to the end token
        if predicted_id == previous_length + 2:
            return tf.squeeze(output, axis=0), attention_weights

        # concatenate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)
    return tf.squeeze(output, axis=0), attention_weights

In [65]:
data_test = tf.data.Dataset.from_tensor_slices((X_tensor_test, Y_tensor_test)).shuffle(len(X_tensor_test))
data_test = data_test.batch(1, drop_remainder=True)

In [66]:
'''Y_true = []
Y_pred = []
correctCount = 0

idx = 0
for(X_test_batch, Y_test_batch) in iter(data_test):
    idx += 1
    if idx % 10 == 0:
        print(f'Samples tested: {idx}, Correctly solved: {correctCount}')
    ground_truth_expression = ''
    for i in Y_test_batch.numpy()[0]:
        if i not in [0, previous_length + 1, previous_length + 2]:
            ground_truth_expression += (Y_lang_tokenizer.index_word[i] + ' ')

    Y_true.append([ground_truth_expression.split(' ')[:-1]])

    prediction, attention_weights = evaluate_testset(X_test_batch)
    predicted_expression = [Y_lang_tokenizer.index_word[i] \
                            for i in list(prediction.numpy()) \
                            if (i < len(Y_lang_tokenizer.word_index) and i not in [0, previous_length + 1, previous_length + 2])]
    Y_pred.append(predicted_expression)
    if ground_truth_expression.split(' ')[:-1] == predicted_expression:
        correctCount += 1'''

"Y_true = []\nY_pred = []\ncorrectCount = 0\n\nidx = 0\nfor(X_test_batch, Y_test_batch) in iter(data_test):\n    idx += 1\n    if idx % 10 == 0:\n        print(f'Samples tested: {idx}, Correctly solved: {correctCount}')\n    ground_truth_expression = ''\n    for i in Y_test_batch.numpy()[0]:\n        if i not in [0, previous_length + 1, previous_length + 2]:\n            ground_truth_expression += (Y_lang_tokenizer.index_word[i] + ' ')\n\n    Y_true.append([ground_truth_expression.split(' ')[:-1]])\n\n    prediction, attention_weights = evaluate_testset(X_test_batch)\n    predicted_expression = [Y_lang_tokenizer.index_word[i]                             for i in list(prediction.numpy())                             if (i < len(Y_lang_tokenizer.word_index) and i not in [0, previous_length + 1, previous_length + 2])]\n    Y_pred.append(predicted_expression)\n    if ground_truth_expression.split(' ')[:-1] == predicted_expression:\n        correctCount += 1"

In [67]:
'''print(len(Y_true), len(Y_pred))'''

'print(len(Y_true), len(Y_pred))'

***RESULTS***

In [68]:
'''print('Corpus BLEU score of the model: ', corpus_bleu(Y_true, Y_pred))'''

"print('Corpus BLEU score of the model: ', corpus_bleu(Y_true, Y_pred))"

In [69]:
'''print(f'Accuracy of the model: {(correctCount/len(X_tensor_test))*100}%')'''

"print(f'Accuracy of the model: {(correctCount/len(X_tensor_test))*100}%')"

In [70]:
sample_problem = ' '.join([X_lang_tokenizer.index_word[i] \
                           for i in X_tensor_test[70] \
                           if i not in [0, len(X_lang_tokenizer.word_index)+1, len(X_lang_tokenizer.word_index)+2]])

In [71]:
sample_problem

'john wants to share some plum among 3 0 friends . if each friend get 3 3 plum , then how many plum john would have ?'

In [72]:
sample_problem

'john wants to share some plum among 3 0 friends . if each friend get 3 3 plum , then how many plum john would have ?'

In [73]:
solve(sample_problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

Input: john wants to share some plum among 3 0 friends . if each friend get 3 3 plum , then how many plum john would have ?
Predicted translation: x = 3 3 * 3 0


In [74]:
solve('A painter needed to paint 12 rooms in a building. Each room takes 7 hours to paint.\
 If he already painted 5 rooms, how much longer will he take to paint the rest?'\
 , plot='', plot_Attention_Weights=False)

Input: A painter needed to paint 12 rooms in a building. Each room takes 7 hours to paint. If he already painted 5 rooms, how much longer will he take to paint the rest?
Predicted translation: x = 7 . * ( 1 2 . 0 - 5 . 0 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )


In [75]:
solve('Rachel bought two coloring books. One had 23 pictures and the other had 32. After one week she had colored 19 of the pictures. How many pictures does she still have to color?', plot='', plot_Attention_Weights=False)

Input: Rachel bought two coloring books. One had 23 pictures and the other had 32. After one week she had colored 19 of the pictures. How many pictures does she still have to color?
Predicted translation: x = ( 3 3 3 . 0 - 1 3 . 0 ) ) ) ) ) )


In [76]:
'''!pip install PySide6'''


'!pip install PySide6'

In [88]:
#MAIN UI
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont
import re

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()


    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Apply dark background color to the entire application
        self.setStyleSheet("background-color: #333333; color: white;")

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        # Apply dark style to the Next button
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        # Style the input and output boxes
        self.style_text_box(self.input_text)
        self.style_text_box(self.generated_equation)
        self.style_text_box(self.solved_equation)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        # Apply dark style to the Generate button
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))
        self.solve_button.clicked.connect(self.solveEquation)  # Connect the button to the solveEquation method

        # Apply dark style to the Solve button
        self.style_button(self.solve_button)

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        # Apply dark style to the main layout
        main_layout.setContentsMargins(20, 20, 20, 20)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the "Generated Equation" text box
        self.generated_equation.setPlainText(predicted_translation)

    def solveEquation(self):
        # Get the predicted translation from the "Generated Equation" text box
        predicted_translation = self.generated_equation.toPlainText()

        # Split the equation by '=' to get the variable and the expression
        parts = predicted_translation.split('=')
        if len(parts) == 2:
            variable, expression = parts[0].strip(), parts[1].strip()

            # Ensure the expression follows BODMAS rules by evaluating it using custom code
            result = self.evaluate_expression(expression)

            # Display the result in the "Solved Equation" text box
            self.solved_equation.setPlainText(f"{variable} = {result}")
        else:
            self.solved_equation.setPlainText("Invalid Equation")

    def evaluate_expression(self, expression):
        try:
            # Remove spaces and evaluate the expression
            expression = expression.replace(" ", "")
            # Replace 'x' with 'X' for better compatibility
            expression = expression.replace('x', 'X')
            # Evaluate the expression
            result = eval(expression)
            return result
        except Exception as e:
            return "Invalid Expression"

    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation

    def style_button(self, button):
        # Apply dark style to the buttons
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 10px 20px; border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

    def style_text_box(self, text_box):
        # Apply dark style to the text boxes
        text_box.setStyleSheet("QTextEdit { background-color: #444444; color: white; border: 1px solid #666666; border-radius: 5px; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


SystemExit: 0

In [77]:
'''import sys
from PySide6.QtWidgets import QApplication, QMainWindow, QPushButton, QLineEdit, QTextBrowser, QLabel

def run_pyside_app():
    app = QApplication.instance()  # Check if QApplication already exists
    if not app:
        app = QApplication(sys.argv)  # Create a new QApplication if it doesn't exist

    window = QMainWindow()
    window.setWindowTitle("MATH AI: Word Problem Equation Generator")
    window.setGeometry(100, 100, 600, 400)  # Adjust the window size

    # Stylesheet for styling the UI
    style = """
        QLabel {
            font-size: 14px;
        }
        QLineEdit, QTextBrowser {
            font-size: 16px;
            padding: 5px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        QPushButton {
            font-size: 16px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            padding: 10px;
        }
        QPushButton:hover {
            background-color: #0056b3;
        }
    """
    window.setStyleSheet(style)

    input_label = QLabel("Enter the word problem here:", window)
    input_label.setGeometry(50, 50, 500, 30)

    input_field = QLineEdit(window)
    input_field.setGeometry(50, 90, 500, 40)  # Adjust field position and size

    result_label = QLabel("Generated Output:", window)
    result_label.setGeometry(50, 150, 500, 30)

    result_area = QTextBrowser(window)
    result_area.setGeometry(50, 190, 500, 100)  # Adjust area position and size

    generate_button = QPushButton("Generate", window)
    generate_button.setGeometry(250, 310, 100, 40)  # Adjust button position

    def generate_clicked():
        sample_problem = input_field.text()
        result = solve(sample_problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)
        result_area.setPlainText(result)

    generate_button.clicked.connect(generate_clicked)

    window.show()

    sys.exit(app.exec_())

# Run the PySide app
run_pyside_app()'''


'import sys\nfrom PySide6.QtWidgets import QApplication, QMainWindow, QPushButton, QLineEdit, QTextBrowser, QLabel\n\ndef run_pyside_app():\n    app = QApplication.instance()  # Check if QApplication already exists\n    if not app:\n        app = QApplication(sys.argv)  # Create a new QApplication if it doesn\'t exist\n\n    window = QMainWindow()\n    window.setWindowTitle("MATH AI: Word Problem Equation Generator")\n    window.setGeometry(100, 100, 600, 400)  # Adjust the window size\n\n    # Stylesheet for styling the UI\n    style = """\n        QLabel {\n            font-size: 14px;\n        }\n        QLineEdit, QTextBrowser {\n            font-size: 16px;\n            padding: 5px;\n            border: 1px solid #ccc;\n            border-radius: 5px;\n        }\n        QPushButton {\n            font-size: 16px;\n            background-color: #007bff;\n            color: white;\n            border: none;\n            border-radius: 5px;\n            padding: 10px;\n        }\n 

In [78]:
'''import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        welcome_layout = QVBoxLayout()
        welcome_layout.addWidget(self.welcome_label)
        welcome_layout.addWidget(self.welcome_next_button)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_text = QTextEdit()
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solved_equation = QTextEdit()

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.clicked.connect(self.generateEquation)

        self.solve_button = QPushButton("Solve Equation")

        main_layout = QVBoxLayout()
        main_layout.addWidget(self.input_text)
        main_layout.addWidget(self.generated_equation)
        main_layout.addWidget(self.solved_equation)
        main_layout.addWidget(self.generate_button)
        main_layout.addWidget(self.solve_button)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the Generated Equation area
        self.generated_equation.setPlainText(predicted_translation)

    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

'import sys\nfrom PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel\n\nclass MathAIApp(QWidget):\n    def __init__(self):\n        super().__init__()\n        self.initUI()\n\n    def initUI(self):\n        self.setWindowTitle(\'MATH AI: Word Problems Solved\')\n        self.setGeometry(100, 100, 800, 600)\n\n        # Welcome Screen\n        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")\n        self.welcome_next_button = QPushButton("Next")\n        self.welcome_next_button.clicked.connect(self.showMainScreen)\n\n        welcome_layout = QVBoxLayout()\n        welcome_layout.addWidget(self.welcome_label)\n        welcome_layout.addWidget(self.welcome_next_button)\n\n        self.welcome_screen = QWidget()\n        self.welcome_screen.setLayout(welcome_layout)\n\n        # Main Screen\n        self.input_text = QTextEdit()\n        self.generated_equation = QTextEdit()\n        self.generated_equation.setReadOnly(True)\

In [79]:
'''import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont  # Import QFont from PyQt5.QtGui


class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()  # Corrected super() call
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the Generated Equation area
        self.generated_equation.setPlainText(predicted_translation)

    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

'import sys\nfrom PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel\nfrom PyQt5.QtCore import Qt\nfrom PyQt5.QtGui import QFont  # Import QFont from PyQt5.QtGui\n\n\nclass MathAIApp(QWidget):\n    def __init__(self):\n        super().__init__()  # Corrected super() call\n        self.initUI()\n\n    def initUI(self):\n        self.setWindowTitle(\'MATH AI: Word Problems Solved\')\n        self.setGeometry(100, 100, 800, 600)\n\n        # Welcome Screen\n        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")\n        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))\n        self.welcome_next_button = QPushButton("Next")\n        self.welcome_next_button.setFont(QFont("Arial", 12))\n        self.welcome_next_button.clicked.connect(self.showMainScreen)\n\n        welcome_layout = QVBoxLayout()\n        welcome_layout.addStretch(1)\n        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.Alig

In [80]:
'''import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        # Apply styles to the Next button
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        # Apply styles to the Generate button
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the Generated Equation area
        self.generated_equation.setPlainText(predicted_translation)

    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation

    def style_button(self, button):
        # Apply Bootstrap-style button styling
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 10px 20px; border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

'import sys\nfrom PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel\nfrom PyQt5.QtCore import Qt\nfrom PyQt5.QtGui import QFont\n\nclass MathAIApp(QWidget):\n    def __init__(self):\n        super().__init__()\n        self.initUI()\n\n    def initUI(self):\n        self.setWindowTitle(\'MATH AI: Word Problems Solved\')\n        self.setGeometry(100, 100, 800, 600)\n\n        # Welcome Screen\n        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")\n        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))\n        self.welcome_next_button = QPushButton("Next")\n        self.welcome_next_button.setFont(QFont("Arial", 12))\n        self.welcome_next_button.clicked.connect(self.showMainScreen)\n\n        # Apply styles to the Next button\n        self.style_button(self.welcome_next_button)\n\n        welcome_layout = QVBoxLayout()\n        welcome_layout.addStretch(1)\n        welcome_layout.addWidget(self.welcome_l

In [81]:
'''import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Apply dark background color to the entire application
        self.setStyleSheet("background-color: #333333; color: white;")

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        # Apply dark style to the Next button
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        # Apply dark style to the Generate button
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))

        # Apply dark style to the Solve button
        self.style_button(self.solve_button)

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        # Apply dark style to the main layout
        main_layout.setContentsMargins(20, 20, 20, 20)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the Generated Equation area
        self.generated_equation.setPlainText(predicted_translation)

    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation

    def style_button(self, button):
        # Apply dark style to the buttons
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 10px 20px; border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

'import sys\nfrom PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel\nfrom PyQt5.QtCore import Qt\nfrom PyQt5.QtGui import QFont\n\nclass MathAIApp(QWidget):\n    def __init__(self):\n        super().__init__()\n        self.initUI()\n\n    def initUI(self):\n        self.setWindowTitle(\'MATH AI: Word Problems Solved\')\n        self.setGeometry(100, 100, 800, 600)\n\n        # Apply dark background color to the entire application\n        self.setStyleSheet("background-color: #333333; color: white;")\n\n        # Welcome Screen\n        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")\n        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))\n        self.welcome_next_button = QPushButton("Next")\n        self.welcome_next_button.setFont(QFont("Arial", 12))\n        self.welcome_next_button.clicked.connect(self.showMainScreen)\n\n        # Apply dark style to the Next button\n        self.style_button(self.welcom

In [82]:
'''#IMPORTANT UI

import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Apply dark background color to the entire application
        self.setStyleSheet("background-color: #333333; color: white;")

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        # Apply dark style to the Next button
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        # Style the input and output boxes
        self.style_text_box(self.input_text)
        self.style_text_box(self.generated_equation)
        self.style_text_box(self.solved_equation)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        # Apply dark style to the Generate button
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))

        # Apply dark style to the Solve button
        self.style_button(self.solve_button)

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        # Apply dark style to the main layout
        main_layout.setContentsMargins(20, 20, 20, 20)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the Generated Equation area
        self.generated_equation.setPlainText(predicted_translation)

    
    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation
    

    def style_button(self, button):
        # Apply dark style to the buttons
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 10px 20px; border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

    def style_text_box(self, text_box):
        # Apply dark style to the text boxes
        text_box.setStyleSheet("QTextEdit { background-color: #444444; color: white; border: 1px solid #666666; border-radius: 5px; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

'#IMPORTANT UI\n\nimport sys\nfrom PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel\nfrom PyQt5.QtCore import Qt\nfrom PyQt5.QtGui import QFont\n\nclass MathAIApp(QWidget):\n    def __init__(self):\n        super().__init__()\n        self.initUI()\n\n    def initUI(self):\n        self.setWindowTitle(\'MATH AI: Word Problems Solved\')\n        self.setGeometry(100, 100, 800, 600)\n\n        # Apply dark background color to the entire application\n        self.setStyleSheet("background-color: #333333; color: white;")\n\n        # Welcome Screen\n        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")\n        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))\n        self.welcome_next_button = QPushButton("Next")\n        self.welcome_next_button.setFont(QFont("Arial", 12))\n        self.welcome_next_button.clicked.connect(self.showMainScreen)\n\n        # Apply dark style to the Next button\n        self.style_b

In [ ]:
'''#Main UI
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Apply dark background color to the entire application
        self.setStyleSheet("background-color: #333333; color: white;")

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        # Apply dark style to the Next button
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        # Style the input and output boxes
        self.style_text_box(self.input_text)
        self.style_text_box(self.generated_equation)
        self.style_text_box(self.solved_equation)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        # Apply dark style to the Generate button
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))

        # Apply dark style to the Solve button
        self.style_button(self.solve_button)

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        # Apply dark style to the main layout
        main_layout.setContentsMargins(20, 20, 20, 20)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)

        # Display the predicted translation in the Generated Equation area
        self.generated_equation.setPlainText(predicted_translation)

    
    def solve(self, problem, plot, plot_Attention_Weights):
        # Your solve logic
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        # Update the UI with the predicted translation
        self.generated_equation.setPlainText(predicted_translation)

        # If needed, add logic for attention weight plotting here
        # plot_attention_weights(attention_weights, problem, prediction, plot)

        return predicted_translation
    

    def style_button(self, button):
        # Apply dark style to the buttons
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 8px 16px; border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

    def style_text_box(self, text_box):
        # Apply dark style to the text boxes
        text_box.setStyleSheet("QTextEdit { background-color: #444444; color: white; border: 1px solid #666666; border-radius: 5px; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''
    
    
    

In [ ]:
'''import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont
import re
import math

class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)

        # Apply dark background color to the entire application
        self.setStyleSheet("background-color: #333333; color: white;")

        # Welcome Screen
        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)

        # Apply dark style to the Next button
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)

        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        # Main Screen
        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)

        # Style the input and output boxes
        self.style_text_box(self.input_text)
        self.style_text_box(self.generated_equation)
        self.style_text_box(self.solved_equation)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)

        # Apply dark style to the Generate button
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))
        self.solve_button.clicked.connect(self.solveEquation)  # Connect the button to the solveEquation method

        # Apply dark style to the Solve button
        self.style_button(self.solve_button)

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        # Apply dark style to the main layout
        main_layout.setContentsMargins(20, 20, 20, 20)

        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        # Stack both screens
        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        # Get the user input problem
        problem = self.input_text.toPlainText()

        # Call the solve function and capture the output
        predicted_translation = self.solve(problem)

        # Display the generated equation in the "Generated Equation" text box
        self.generated_equation.setPlainText(predicted_translation)

    def solveEquation(self):
        # Get the generated equation
        generated_equation = self.generated_equation.toPlainText()

        # Call the solve_equation function and capture the output
        solved_equation = self.solve_equation(generated_equation)

        # Display the solved equation in the "Solved Equation" text box
        self.solved_equation.setPlainText(solved_equation)

    def solve_equation(self, equation):
        try:
            # Split the equation by '=' to get the variable and the expression
            variable, expression = equation.split('=')
            variable = variable.strip()
            expression = expression.strip()

            # Ensure the expression follows BODMAS rules by evaluating it using custom code
            result = self.evaluate_expression(expression)
            return f"{variable} = {result}"
        except Exception as e:
            return f"Error: {str(e)}"

    def evaluate_expression(self, expression):
        try:
            # Split the expression by spaces and evaluate it following BODMAS rules
            parts = re.findall(r'(\d+|\+|\-|\*|/|\(|\))', expression)
            cleaned_expression = ' '.join(parts)
            result = eval(cleaned_expression)
            return result
        except Exception as e:
            return f"Invalid Expression"

    def solve(problem, plot='', plot_Attention_Weights=False):
        prediction, attention_weights = evaluate(problem)
        predicted_expression = [Y_lang_tokenizer.index_word[i] \
                              for i in list(prediction.numpy()) \
                              if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)

        print(f'Input: {input_text}')
        print(f'Predicted translation: {predicted_translation}')



        if plot_Attention_Weights:
            plot_attention_weights(attention_weights, problem, prediction, plot)

        return

    def style_button(self, button):
        # Apply dark style to the buttons
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 8px 16px; border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

    def style_text_box(self, text_box):
        # Apply dark style to the text boxes
        text_box.setStyleSheet("QTextEdit { background-color: #444444; color: white; border: 1px solid #666666; border-radius: 5px; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''

In [ ]:
'''import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLabel
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QFont
import re
import sympy


class MathAIApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('MATH AI: Word Problems Solved')
        self.setGeometry(100, 100, 800, 600)
        self.setStyleSheet("background-color: #333333; color: white;")

        self.welcome_label = QLabel("Welcome To MATH AI: Word Problems Solved")
        self.welcome_label.setFont(QFont("Arial", 16, QFont.Bold))
        self.welcome_next_button = QPushButton("Next")
        self.welcome_next_button.setFont(QFont("Arial", 12))
        self.welcome_next_button.clicked.connect(self.showMainScreen)
        self.style_button(self.welcome_next_button)

        welcome_layout = QVBoxLayout()
        welcome_layout.addStretch(1)
        welcome_layout.addWidget(self.welcome_label, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addWidget(self.welcome_next_button, alignment=Qt.AlignmentFlag.AlignCenter)
        welcome_layout.addStretch(1)
        self.welcome_screen = QWidget()
        self.welcome_screen.setLayout(welcome_layout)

        self.input_label = QLabel("Input Word Problem:")
        self.input_text = QTextEdit()
        self.generated_label = QLabel("Generated Equation:")
        self.generated_equation = QTextEdit()
        self.generated_equation.setReadOnly(True)
        self.solve_label = QLabel("Solved Equation:")
        self.solved_equation = QTextEdit()
        self.solved_equation.setReadOnly(True)
        self.style_text_box(self.input_text)
        self.style_text_box(self.generated_equation)
        self.style_text_box(self.solved_equation)

        self.generate_button = QPushButton("Generate Equation")
        self.generate_button.setFont(QFont("Arial", 12))
        self.generate_button.clicked.connect(self.generateEquation)
        self.style_button(self.generate_button)

        self.solve_button = QPushButton("Solve Equation")
        self.solve_button.setFont(QFont("Arial", 12))
        self.solve_button.clicked.connect(self.solveEquation)
        self.style_button(self.solve_button)

        main_layout = QVBoxLayout()
        input_layout = QVBoxLayout()
        input_layout.addWidget(self.input_label)
        input_layout.addWidget(self.input_text)
        generated_layout = QVBoxLayout()
        generated_layout.addWidget(self.generated_label)
        generated_layout.addWidget(self.generated_equation)
        solve_layout = QVBoxLayout()
        solve_layout.addWidget(self.solve_label)
        solve_layout.addWidget(self.solved_equation)

        main_layout.addLayout(input_layout)
        main_layout.addWidget(self.generate_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(generated_layout)
        main_layout.addWidget(self.solve_button, alignment=Qt.AlignmentFlag.AlignCenter)
        main_layout.addLayout(solve_layout)

        main_layout.setContentsMargins(20, 20, 20, 20)
        self.main_screen = QWidget()
        self.main_screen.setLayout(main_layout)
        self.main_screen.hide()

        self.stack = QVBoxLayout()
        self.stack.addWidget(self.welcome_screen)
        self.stack.addWidget(self.main_screen)

        self.setLayout(self.stack)

    def showMainScreen(self):
        self.welcome_screen.hide()
        self.main_screen.show()

    def generateEquation(self):
        problem = self.input_text.toPlainText()
        predicted_translation = self.solve(problem, plot='decoder_layer4_block2', plot_Attention_Weights=False)
        self.generated_equation.setPlainText(predicted_translation)

    def solveEquation(self):
        equation = self.generated_equation.toPlainText()
        result = self.solve_equation(equation)
        self.solved_equation.setPlainText(result)

    def solve_equation(self, equation):
        # Remove unnecessary spaces from the equation
        cleaned_equation = re.sub(r'\s+', '', equation)

        try:
            x = sympy.symbols('x')
            equation_result = sympy.solve(sympy.Eq(x, sympy.sympify(cleaned_equation)), x)

            if equation_result:
                result = equation_result[0]
            else:
                result = "No solution found"
        except Exception as e:
            result = str(e)

        return str(result)

    def solve(self, problem, plot, plot_Attention_Weights):
        prediction, attention_weights = evaluate(problem)  # Call your evaluate function
        predicted_expression = [Y_lang_tokenizer.index_word[i] for i in list(prediction.numpy())
                                if (i < len(Y_lang_tokenizer.word_index) and i not in [0, 46, 47])]
        input_text = problem
        predicted_translation = ' '.join(predicted_expression)
        self.generated_equation.setPlainText(predicted_translation)

    def style_button(self, button):
        button.setStyleSheet("QPushButton { background-color: #007BFF; color: white; padding: 8px 16px; "
                             "border: none; border-radius: 5px; }"
                             "QPushButton:hover { background-color: #0056b3; }")

    def style_text_box(self, text_box):
        text_box.setStyleSheet("QTextEdit { background-color: #444444; color: white; border: 1px solid #666666; "
                               "border-radius: 5px; }")

def main():
    app = QApplication(sys.argv)
    math_ai = MathAIApp()
    math_ai.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()
'''